In [2]:
import re
import pandas as pd
from textblob import TextBlob

In [3]:
def isNaN(num):
    return num != num

In [6]:
df = pd.read_csv('humans.csv', sep=';', index_col='id', usecols = ['id', 'source', 'is_retweet', 'hashtags', 'lemma_filter'])

In [7]:
df0 = df[df['is_retweet'] == False].sample(frac=0.1)
df0_1 = df0[df0.apply(lambda x: (isNaN(x['lemma_filter'])==False) and (isNaN(x['hashtags'])==False), axis=1)]
df0_1['hashtags_list'] = df0_1.apply(lambda x: [i for i in set(re.findall(r"[\w']+", x['hashtags'].lower())) if len(i) > 1], axis=1)
df1 = df0_1[df0_1.apply(lambda x: len(x['hashtags_list']) > 0, axis=1)]

c:\users\ipsita\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
df1.head(5)

,source,is_retweet,hashtags,lemma_filter,hashtags_list
id,,,,,
251177,instagram,False,"#iVoted, #OnDeck, #RunUp, #100each$, #Vert, #A...",get sticker free vote ! ai n't 💀👎,"[100each, ygb, ondeck, ig, ny, lcorps, ca, dom..."
217970,instagram,False,"#25novembre2018, #25novembre, #metoo, #quellav...",domenica sarò anch'io sul palco come oltreleco...,"[metoo, quellavoltache, 25novembre2018, 25nove..."
87429,instagram,False,"#Uniformesfutbol., #uniformesdefutbol, #unifor...",niformesfutbol . inst niformesfutboll . . . . ...,"[happy, nortedesantander, bogota, cali, futbol..."
27586,instagram,False,"#performanceart, #video, #videoart, #emerginga...",new video work make wrong side bed •/,"[erase, hide, artist, femaleartist, interdisci..."
296622,instagram,False,"#nin, #nineinchnails, #trentreznor, #downinit,...",find long lose stash ... ..,"[coolshit, trentreznor, metoo, fordmodels, nin..."


In [9]:
df2 = df1
df2['sentiment_polarity'] = df2.apply(lambda x: TextBlob(x['lemma_filter']).sentiment[0], axis=1)
df2['sentiment'] = df2.apply(lambda x: 'pos' if x['sentiment_polarity'] > 0 else 'neg' if x['sentiment_polarity'] < 0 else 'neutral', axis=1)

c:\users\ipsita\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\ipsita\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
df2.head(5)

,source,is_retweet,hashtags,lemma_filter,hashtags_list,sentiment_polarity,sentiment
id,,,,,,,
251177,instagram,False,"#iVoted, #OnDeck, #RunUp, #100each$, #Vert, #A...",get sticker free vote ! ai n't 💀👎,"[100each, ygb, ondeck, ig, ny, lcorps, ca, dom...",0.500000,pos
217970,instagram,False,"#25novembre2018, #25novembre, #metoo, #quellav...",domenica sarò anch'io sul palco come oltreleco...,"[metoo, quellavoltache, 25novembre2018, 25nove...",0.000000,neutral
87429,instagram,False,"#Uniformesfutbol., #uniformesdefutbol, #unifor...",niformesfutbol . inst niformesfutboll . . . . ...,"[happy, nortedesantander, bogota, cali, futbol...",0.000000,neutral
27586,instagram,False,"#performanceart, #video, #videoart, #emerginga...",new video work make wrong side bed •/,"[erase, hide, artist, femaleartist, interdisci...",-0.181818,neg
296622,instagram,False,"#nin, #nineinchnails, #trentreznor, #downinit,...",find long lose stash ... ..,"[coolshit, trentreznor, metoo, fordmodels, nin...",-0.050000,neg


In [11]:
NUM = 50  #UM of most popular hashtags
# # Only selecting positive and negative sentiment
# # df3 = df2[df2['sentiment'] != 'neutral'].set_index(['sentiment'])['hashtags_list'].apply(pd.Series).stack()

# Select all three sentiments
df3 = df2.set_index(['sentiment'])['hashtags_list'].apply(pd.Series).stack()
df3 = df3.reset_index()
df3.columns = ['sentiment', 'sample_num','hashtag']
df3_1 = df3.groupby(['hashtag', 'sentiment']).size().reset_index(name='counts')
df3_2 = df3.groupby(['hashtag']).size().reset_index(name='counts')
df3_merged = pd.merge(df3_1, df3_2, on='hashtag', how='inner')
result = df3_merged[df3_merged['counts_y'] > NUM] #df3_merged.nlargest(NUM, 'counts_y') 
result['probe'] = result.apply(lambda x: float(x['counts_x'])/x['counts_y'], axis=1)
# result = result[result.apply(lambda x: 'metoo' not in x['hashtag'], axis=1)]

# # result
# #
hashtag_dic = {}
for index, row in result.iterrows():
    hashtag_dic[row['sentiment'], row['hashtag']] = row['probe']

# 2019womensmarch

# #
hashtags_set = set(result['hashtag'].values)
len(hashtags_set)

c:\users\ipsita\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


1042

In [13]:
hashtags_set

{'red',
 'warrior',
 'revolution',
 'relation',
 'climatechange',
 'dumptrump',
 'metoodoll',
 'dailymemes',
 '2019',
 'sex',
 'spain',
 '16daysofactivism',
 'bossbabe',
 'loveposts',
 'lgbtqcommunity',
 'mind',
 'safety',
 'selflove',
 'staystrong',
 'hate',
 'environment',
 'queen',
 'speakout',
 'miracomonosponemos',
 'highendfashion',
 'violenceagainstwomen',
 'businesswoman',
 'beach',
 'daughter',
 'metoo',
 'wordporn',
 'advocate',
 'propaganda',
 'feminismisforeverybody',
 'seattle',
 'stopkavanaugh',
 'recovery',
 'violenciadegenero',
 'guncontrol',
 'divorce',
 'blackgirlsrock',
 'freedom',
 'author',
 'music',
 'neveragain',
 'healthy',
 'creativity',
 'gender',
 'fortnitememes',
 'magazine',
 'bosswoman',
 'truth',
 'lifestyle',
 'romantic',
 'happiness',
 'nomeansno',
 'bewell',
 'business',
 'writersofig',
 'womensmarch2019',
 'impeachment',
 'blog',
 'digitalart',
 'friendship',
 'filthyfrank',
 'heart',
 'civilrights',
 'relatable',
 'dc',
 'graffiti',
 'immigrants',
 '

In [14]:
df1['idx'] = df1.index
tdf = df1.set_index(['idx'])['hashtags_list'].apply(pd.Series).stack()
tdf = tdf.reset_index()
tdf.columns = ['idx', 'sample_num','hashtag']
ht_dic = {}
for index, row in tdf.iterrows():
    if row['hashtag'] in ht_dic:
        ht_dic[row['hashtag']].add(row['idx'])
    else:
        ht_dic[row['hashtag']] = set([row['idx']])

c:\users\ipsita\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
ht_dic

{'sexology': {203541},
 'charliesheen': {331911},
 'french': {13542,
  21573,
  82093,
  119574,
  121670,
  159885,
  164528,
  164548,
  180075,
  216707,
  243260,
  324363,
  326866},
 'bandhva': {280342},
 'raiseboystobemen': {6356,
  9006,
  33258,
  39643,
  43100,
  50076,
  77884,
  79033,
  98553,
  200505,
  243922,
  293143},
 'taichung': {262348},
 'abetterworld': {123869, 267599, 267603},
 'dejaquenieve': {98907},
 'punisherslemc': {318534},
 'anawarrior': {19573, 42632},
 'violconjugal': {206883},
 'amateurs_bnw': {207784},
 'classyoutfit': {336676, 337746},
 '憲法カフェ': {301156},
 'singlefather': {218012},
 '지지합니다': {81126, 81128, 81165, 82301, 176785, 176903, 181752},
 'heartbeatbill': {245125},
 'punjabiwedding': {182870},
 'baaintheair': {29442},
 'queen': {405,
  9330,
  26139,
  26971,
  27046,
  29567,
  29996,
  31795,
  32943,
  33193,
  39366,
  39710,
  42935,
  43133,
  46552,
  47403,
  53204,
  53448,
  55240,
  62166,
  62832,
  63720,
  63805,
  66718,
  676

In [11]:
import itertools
SI = {}
for (i, j) in itertools.combinations(list(hashtags_set), 2):
    tmp = 0
    if i in ht_dic:
        i_cnt = len(ht_dic[i])
        tmp += 1
    if j in ht_dic:
        j_cnt = len(ht_dic[j])
        tmp += 1
    if tmp == 2:
        i_j_cnt = len(ht_dic[i].intersection(ht_dic[j]))
    if i_j_cnt > 0:
        SI[i, j] = float(i_j_cnt)/(i_cnt + j_cnt)

In [12]:
import pickle
with open('hashtag.pickle', 'wb') as handle:
    pickle.dump([list(hashtags_set), hashtag_dic, SI], handle, protocol=pickle.HIGHEST_PROTOCOL)